In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Flatten, Input
from gensim.models import Word2Vec

In [4]:
import gensim
from gensim.models import Word2Vec

In [5]:
jokes = pd.read_csv("../input/tokenized-mil-jokes/million_jokes_tokenized.csv")
jokes

/tmp/ipykernel_29/219361694.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  jokes = pd.read_csv("../input/tokenized-mil-jokes/million_jokes_tokenized.csv")


,score,joke,tokens,tokenized_joke
0,2,i am soooo glad i'm not circumcised!my corona ...,"['i', 'am', 'soooo', 'glad', 'i', ""'m"", 'not',...","['i', 'am', 'soooo', 'glad', 'i', ""'m"", 'not',..."
1,9,did you know google now has a platform for rec...,"['did', 'you', 'know', 'google', 'now', 'has',...","['did', 'you', 'know', 'google', 'now', 'has',..."
2,15,what is the difference between my wife and my ...,"['what', 'is', 'the', 'difference', 'between',...","['what', 'is', 'the', 'difference', 'between',..."
3,9,my last joke for now.,"['my', 'last', 'joke', 'for', 'now.[removed', ...","['my', 'last', 'joke', 'for', 'now', '.']"
4,134,the nintendo 64 turns 18 this week...,"['the', 'nintendo', '64', 'turns', '18', 'this...","['the', 'nintendo', '64', 'turns', '18', 'this..."
...,...,...,...,...
938409,0,with zyan malik leaving 1d..*zyan malik or wha...,"['with', 'zyan', 'malik', 'leaving', '1d', '.....","['with', 'zyan', 'malik', 'leaving', '1d', '....."
938410,1,why did the exterminator go to italy?,"['why', 'did', 'the', 'exterminator', 'go', 't...","['why', 'did', 'the', 'exterminator', 'go', 't..."
938411,0,what did arnold schwarzenegger say when invite...,"['what', 'did', 'arnold', 'schwarzenegger', 's...","['what', 'did', 'arnold', 'schwarzenegger', 's..."
938412,87,the moth jokeso a moth goes into a podiatrists...,"['the', 'moth', 'jokeso', 'a', 'moth', 'goes',...","['the', 'moth', 'jokeso', 'a', 'moth', 'goes',..."


In [7]:
jokes = jokes.dropna(subset=['joke'])
jokes = jokes.reset_index(drop=True)

In [10]:
jokes = jokes.dropna(subset=['score'])
jokes = jokes.reset_index(drop=True)

In [13]:
jokes = jokes.dropna(subset=['tokenized_joke'])
jokes = jokes.reset_index(drop=True)

In [20]:
jokes.drop(['tokens'], axis=1, inplace = True)
jokes

,score,joke,tokenized_joke
0,2,i am soooo glad i'm not circumcised!my corona ...,"['i', 'am', 'soooo', 'glad', 'i', ""'m"", 'not',..."
1,9,did you know google now has a platform for rec...,"['did', 'you', 'know', 'google', 'now', 'has',..."
2,15,what is the difference between my wife and my ...,"['what', 'is', 'the', 'difference', 'between',..."
3,9,my last joke for now.,"['my', 'last', 'joke', 'for', 'now', '.']"
4,134,the nintendo 64 turns 18 this week...,"['the', 'nintendo', '64', 'turns', '18', 'this..."
...,...,...,...
938387,0,with zyan malik leaving 1d..*zyan malik or wha...,"['with', 'zyan', 'malik', 'leaving', '1d', '....."
938388,1,why did the exterminator go to italy?,"['why', 'did', 'the', 'exterminator', 'go', 't..."
938389,0,what did arnold schwarzenegger say when invite...,"['what', 'did', 'arnold', 'schwarzenegger', 's..."
938390,87,the moth jokeso a moth goes into a podiatrists...,"['the', 'moth', 'jokeso', 'a', 'moth', 'goes',..."


In [24]:
jokes['score'] = pd.to_numeric(jokes['score'], errors='coerce')
jokes = jokes.dropna(subset=['score'])
jokes['score'] = jokes['score'].astype(int)
jokes

/tmp/ipykernel_29/3581081740.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jokes['score'] = jokes['score'].astype(int)


,score,joke,tokenized_joke
0,2,i am soooo glad i'm not circumcised!my corona ...,"['i', 'am', 'soooo', 'glad', 'i', ""'m"", 'not',..."
1,9,did you know google now has a platform for rec...,"['did', 'you', 'know', 'google', 'now', 'has',..."
2,15,what is the difference between my wife and my ...,"['what', 'is', 'the', 'difference', 'between',..."
3,9,my last joke for now.,"['my', 'last', 'joke', 'for', 'now', '.']"
4,134,the nintendo 64 turns 18 this week...,"['the', 'nintendo', '64', 'turns', '18', 'this..."
...,...,...,...
938387,0,with zyan malik leaving 1d..*zyan malik or wha...,"['with', 'zyan', 'malik', 'leaving', '1d', '....."
938388,1,why did the exterminator go to italy?,"['why', 'did', 'the', 'exterminator', 'go', 't..."
938389,0,what did arnold schwarzenegger say when invite...,"['what', 'did', 'arnold', 'schwarzenegger', 's..."
938390,87,the moth jokeso a moth goes into a podiatrists...,"['the', 'moth', 'jokeso', 'a', 'moth', 'goes',..."


In [26]:
jokes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 938376 entries, 0 to 938391
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   score           938376 non-null  int64 
 1   joke            938376 non-null  object
 2   tokenized_joke  938376 non-null  object
dtypes: int64(1), object(2)
memory usage: 28.6+ MB


In [28]:
tokenized_jokes = jokes['tokenized_joke'].tolist()

In [29]:
w2v_model = Word2Vec(vector_size=200, window=5, sg=1, min_count=1, workers=4)

In [30]:
w2v_model.build_vocab(tokenized_jokes, progress_per=10000)

In [31]:
print("Vocabulary size:", len(w2v_model.wv.key_to_index))

Vocabulary size: 2425


In [32]:
w2v_model.train(tokenized_jokes, total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)

(235669822, 1268096325)

In [34]:
def average_vector(tokens, model):
    # Initialize a list to hold the vectors
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    # If no vectors are found, return a zero vector
    if not vectors:
        return np.zeros(model.vector_size)
    # Calculate the average
    return np.mean(vectors, axis=0)

sz = len(jokes)
joke_vec = []
for i in range(sz):
    tokens = jokes.iloc[i]['tokenized_joke']  # Use .iloc to access the row by position
    joke_vec.append(average_vector(tokens, w2v_model))

    if i % 10000 == 0:
        print(f"Processed {i} jokes")

Processed 0 jokes
Processed 10000 jokes
Processed 20000 jokes
Processed 30000 jokes
Processed 40000 jokes
Processed 50000 jokes
Processed 60000 jokes
Processed 70000 jokes
Processed 80000 jokes
Processed 90000 jokes
Processed 100000 jokes
Processed 110000 jokes
Processed 120000 jokes
Processed 130000 jokes
Processed 140000 jokes
Processed 150000 jokes
Processed 160000 jokes
Processed 170000 jokes
Processed 180000 jokes
Processed 190000 jokes
Processed 200000 jokes
Processed 210000 jokes
Processed 220000 jokes
Processed 230000 jokes
Processed 240000 jokes
Processed 250000 jokes
Processed 260000 jokes
Processed 270000 jokes
Processed 280000 jokes
Processed 290000 jokes
Processed 300000 jokes
Processed 310000 jokes
Processed 320000 jokes
Processed 330000 jokes
Processed 340000 jokes
Processed 350000 jokes
Processed 360000 jokes
Processed 370000 jokes
Processed 380000 jokes
Processed 390000 jokes
Processed 400000 jokes
Processed 410000 jokes
Processed 420000 jokes
Processed 430000 jokes
Pr

In [35]:
print(type(joke_vec))          # Should output: <class 'list'>
print(type(joke_vec[0]))       # Should output: <class 'numpy.ndarray'>
print(len(joke_vec))  

<class 'list'>
<class 'numpy.ndarray'>
938376


In [37]:
joke_vec_df = pd.DataFrame(joke_vec, columns=[f'feature_{i}' for i in range(1, 201)])
joke_vec_df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200
0,0.022757,-0.002372,0.015758,-0.045848,-0.030391,0.028164,-0.001990,-0.015856,-0.019119,0.008749,...,-0.044217,0.031029,-0.078493,-0.092353,0.078087,-0.148973,-0.026629,-0.011713,-0.008545,-0.009422
1,0.024044,-0.000378,0.021938,-0.047440,-0.025839,0.027885,-0.005029,-0.007133,-0.012511,0.007561,...,-0.044732,0.035929,-0.079028,-0.096695,0.072428,-0.139816,-0.029196,-0.008497,-0.008033,-0.002562
2,0.014168,0.000887,-0.000752,-0.044718,-0.024526,0.024671,-0.010457,-0.039224,-0.009665,0.006192,...,-0.043452,0.029735,-0.076716,-0.087395,0.075407,-0.134907,-0.028808,-0.012493,-0.003708,-0.010835
3,0.034438,-0.000132,0.018499,-0.051163,-0.011006,0.027088,-0.013480,-0.020603,-0.007974,0.017944,...,-0.044714,0.033919,-0.084499,-0.084496,0.085220,-0.156643,-0.035098,-0.006974,0.005081,-0.009688
4,0.026842,0.015220,-0.004518,-0.056918,-0.011038,0.035486,-0.000196,-0.017904,-0.012727,0.017914,...,-0.026085,0.035004,-0.076037,-0.077958,0.078446,-0.150950,-0.036798,-0.024185,0.008632,-0.003396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938371,0.021803,-0.006288,0.007996,-0.056848,-0.018954,0.023151,-0.003778,-0.028800,-0.016352,0.017205,...,-0.044380,0.028480,-0.070826,-0.083083,0.080424,-0.147250,-0.030937,-0.009036,-0.005118,-0.013136
938372,0.014199,0.002449,0.020642,-0.048500,-0.024897,0.029123,-0.003884,-0.016504,-0.014601,0.008176,...,-0.039253,0.031398,-0.085969,-0.098966,0.081853,-0.147432,-0.028211,-0.013138,-0.004501,-0.010649
938373,0.016331,0.006630,0.016171,-0.048114,-0.028754,0.027912,0.002133,-0.008664,-0.020083,-0.000305,...,-0.039101,0.031835,-0.081894,-0.099352,0.074275,-0.141356,-0.022532,-0.011313,-0.019001,-0.006524
938374,0.015655,-0.005152,0.014912,-0.054261,-0.023497,0.025183,-0.004555,-0.022054,-0.014898,0.010168,...,-0.045308,0.029676,-0.075360,-0.092586,0.079784,-0.146592,-0.029108,-0.013204,-0.004499,-0.008895


In [38]:
jokes = pd.concat([jokes.reset_index(drop=True), joke_vec_df.reset_index(drop=True)], axis=1)
jokes

,score,joke,tokenized_joke,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200
0,2,i am soooo glad i'm not circumcised!my corona ...,"['i', 'am', 'soooo', 'glad', 'i', ""'m"", 'not',...",0.022757,-0.002372,0.015758,-0.045848,-0.030391,0.028164,-0.001990,...,-0.044217,0.031029,-0.078493,-0.092353,0.078087,-0.148973,-0.026629,-0.011713,-0.008545,-0.009422
1,9,did you know google now has a platform for rec...,"['did', 'you', 'know', 'google', 'now', 'has',...",0.024044,-0.000378,0.021938,-0.047440,-0.025839,0.027885,-0.005029,...,-0.044732,0.035929,-0.079028,-0.096695,0.072428,-0.139816,-0.029196,-0.008497,-0.008033,-0.002562
2,15,what is the difference between my wife and my ...,"['what', 'is', 'the', 'difference', 'between',...",0.014168,0.000887,-0.000752,-0.044718,-0.024526,0.024671,-0.010457,...,-0.043452,0.029735,-0.076716,-0.087395,0.075407,-0.134907,-0.028808,-0.012493,-0.003708,-0.010835
3,9,my last joke for now.,"['my', 'last', 'joke', 'for', 'now', '.']",0.034438,-0.000132,0.018499,-0.051163,-0.011006,0.027088,-0.013480,...,-0.044714,0.033919,-0.084499,-0.084496,0.085220,-0.156643,-0.035098,-0.006974,0.005081,-0.009688
4,134,the nintendo 64 turns 18 this week...,"['the', 'nintendo', '64', 'turns', '18', 'this...",0.026842,0.015220,-0.004518,-0.056918,-0.011038,0.035486,-0.000196,...,-0.026085,0.035004,-0.076037,-0.077958,0.078446,-0.150950,-0.036798,-0.024185,0.008632,-0.003396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938371,0,with zyan malik leaving 1d..*zyan malik or wha...,"['with', 'zyan', 'malik', 'leaving', '1d', '.....",0.021803,-0.006288,0.007996,-0.056848,-0.018954,0.023151,-0.003778,...,-0.044380,0.028480,-0.070826,-0.083083,0.080424,-0.147250,-0.030937,-0.009036,-0.005118,-0.013136
938372,1,why did the exterminator go to italy?,"['why', 'did', 'the', 'exterminator', 'go', 't...",0.014199,0.002449,0.020642,-0.048500,-0.024897,0.029123,-0.003884,...,-0.039253,0.031398,-0.085969,-0.098966,0.081853,-0.147432,-0.028211,-0.013138,-0.004501,-0.010649
938373,0,what did arnold schwarzenegger say when invite...,"['what', 'did', 'arnold', 'schwarzenegger', 's...",0.016331,0.006630,0.016171,-0.048114,-0.028754,0.027912,0.002133,...,-0.039101,0.031835,-0.081894,-0.099352,0.074275,-0.141356,-0.022532,-0.011313,-0.019001,-0.006524
938374,87,the moth jokeso a moth goes into a podiatrists...,"['the', 'moth', 'jokeso', 'a', 'moth', 'goes',...",0.015655,-0.005152,0.014912,-0.054261,-0.023497,0.025183,-0.004555,...,-0.045308,0.029676,-0.075360,-0.092586,0.079784,-0.146592,-0.029108,-0.013204,-0.004499,-0.008895


# Data Modelling

In [39]:
X = jokes.drop(['score', 'joke', 'tokenized_joke'], axis=1)
y = jokes.score

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
X_train_reshaped = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

In [51]:
model = Sequential()
# Input layer
model.add(Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
# First LSTM layer
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
# Second LSTM layer
model.add(LSTM(64, return_sequences=False))  
model.add(Dropout(0.3))
# Dense layers
model.add(Dense(32, activation='relu'))  # Intermediate dense layer
model.add(Dropout(0.3))
# Output layer
model.add(Dense(1, activation='relu'))
# Compile the model
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['mean_squared_error'])
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 1, 128)         │       168,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 219,969 (859.25 KB)

 Trainable params: 219,969 (859.25 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test), epochs=20, batch_size=64)

Epoch 1/20
11730/11730 ━━━━━━━━━━━━━━━━━━━━ 80s 7ms/step - loss: 2371643.0000 - mean_squared_error: 2371643.0000 - val_loss: 2200010.5000 - val_mean_squared_error: 2200010.5000
Epoch 2/20
11730/11730 ━━━━━━━━━━━━━━━━━━━━ 76s 7ms/step - loss: 2271523.0000 - mean_squared_error: 2271523.0000 - val_loss: 2199909.7500 - val_mean_squared_error: 2199909.7500
Epoch 3/20
11730/11730 ━━━━━━━━━━━━━━━━━━━━ 76s 6ms/step - loss: 2319313.0000 - mean_squared_error: 2319313.0000 - val_loss: 2200778.0000 - val_mean_squared_error: 2200778.0000
Epoch 4/20
11730/11730 ━━━━━━━━━━━━━━━━━━━━ 76s 6ms/step - loss: 2254769.2500 - mean_squared_error: 2254769.2500 - val_loss: 2202434.2500 - val_mean_squared_error: 2202434.2500
Epoch 5/20
11730/11730 ━━━━━━━━━━━━━━━━━━━━ 77s 7ms/step - loss: 2341514.0000 - mean_squared_error: 2341514.0000 - val_loss: 2200093.7500 - val_mean_squared_error: 2200093.7500
Epoch 6/20
11730/11730 ━━━━━━━━━━━━━━━━━━━━ 77s 7ms/step - loss: 2207933.5000 - mean_squared_error: 2207933.5000 - 

In [53]:
predictions = model.predict(X_test_reshaped)

5865/5865 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step


In [56]:
import pickle

# Save the model with pickle
with open('humor_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [57]:
w2v_model.save("../working/word2vec_humor_model.model")